In [84]:
#Run with python3

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import datetime

In [90]:
base_url = 'http://gasprices.aaa.com?state='
states= ['AZ', 'NV', 'CA', 'OR', 'WA', 'VA', 'IL']
date=datetime.datetime.now().strftime("%m/%d/%Y")

In [91]:
def fetchPrices(state):
    #setup
    url=base_url+state
    req=Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html_week = urlopen(req).read()
    soup = BeautifulSoup(html_week, 'html.parser')
    
    metroPrices=soup.find("div", {"class": "accordion-prices"})
    h3=metroPrices.select("h3")
    tables=metroPrices.select("table")
    
    #create dataframe
    rowList=[]
    for i in range(len(h3)):
        city=h3[i].text
        entries=tables[i*4+2].select("td")
        regular=entries[1].text[1:]
        mid=entries[3].text[1:]
        premium=entries[5].text[1:]
        diesel=entries[7].text[1:]
        rowList.append((date, state, city, diesel, regular, mid, premium))
    return rowList

In [92]:
rowList=[]
for state in states:
    print('fetching', state)
    rowList+=fetchPrices(state)
columns=['Date', 'State', 'City', 'Diesel', 'Regular', 'Mid', 'Premium']
gas_prices=pd.DataFrame(rowList, columns=columns)

fetching AZ
fetching NV
fetching CA
fetching OR
fetching WA
fetching VA
fetching IL


In [95]:
gas_prices[gas_prices['State']=='IL']

,Date,State,City,Diesel,Regular,Mid,Premium
56,06/04/2016,IL,Bloomington-Normal,2.293,2.395,2.607,2.869
57,06/04/2016,IL,Champaign-Urbana,2.241,2.306,2.534,2.806
58,06/04/2016,IL,Chicago Metro,2.450,2.674,2.988,3.375
59,06/04/2016,IL,Decatur,2.181,2.481,2.665,2.892
60,06/04/2016,IL,Peoria-Pekin,2.290,2.385,2.609,2.902
61,06/04/2016,IL,Rockford,2.380,2.426,2.694,3.067
62,06/04/2016,IL,Springfield,2.159,2.369,2.562,2.835
63,06/04/2016,IL,Davenport-Moline-Rock Island (IL only),2.327,2.309,2.577,2.946
64,06/04/2016,IL,East Saint Louis,2.442,2.474,2.661,2.896
65,06/04/2016,IL,Quincy,2.259,2.399,2.572,2.918


In [89]:
gas_prices.to_csv("aaa-price.csv", index=False)